In [94]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# import seaborn as sns
# import matplotlib.pyplot as plt
# import squarify
# import plotly.express as px

TOP_K = 15

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-94-c8363f98b771>:15: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



In [95]:
dataset = pd.read_csv("./data/Emails.csv")
df = pd.DataFrame(dataset)

df.rename(columns={"MetadataDateSent": "DateSent"}, inplace=True)
df = pd.DataFrame(df[["SenderPersonId","DateSent"]])
df.dropna(inplace = True)

df["DateSent"] = df["DateSent"].astype(str)
df["DateSent"]= df["DateSent"].str.slice(0, 10, 1)
df["DateSent"] = pd.to_datetime(df["DateSent"])

df["SenderPersonId"] = df["SenderPersonId"].astype(str)
df["SenderPersonId"] = df["SenderPersonId"].str.slice(0, -2, 1)

print("First email was on:", df.DateSent.min())
print("Last email was on:", df.DateSent.max())
print("The dataset covers", df.DateSent.max() - df.DateSent.min(), "days")
print("The most active day was",df["DateSent"].value_counts().index[0],
      "with",df["DateSent"].value_counts()[0],"emails")

First email was on: 2008-05-01 00:00:00
Last email was on: 2014-12-14 00:00:00
The dataset covers 2418 days 00:00:00 days
The most active day was 2009-12-23 00:00:00 with 47 emails


In [96]:
# Assegnare ID in persons.csv a dataframe principale, così da poter tradurre gli IDs in nomi
persons = pd.read_csv("./data/Persons.csv")
persons = persons.to_dict()
persons = persons["Name"]
df["SenderFullName"] = None

i = 0
for cell,i in zip(df["SenderPersonId"].items(),range(0,len(df))):
    for key in persons.keys():
        if cell[1] == str(key):
            df["SenderFullName"].iloc[i] = persons[key-1]
            i += 1

/Users/gianlucagiudice/.pyenv/versions/3.8.7/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [97]:
i = 0
x = 0
num = 490
while i < 10:
    for date in df['DateSent'].value_counts(normalize=True)[:num]:
        x += df['DateSent'].value_counts(normalize=True)[i]
        i += 1
print(f"In {num} giorni, cioè il {round(num/int(df['DateSent'].count()),2)*100}% del totale dei "
      f"giorni, si è concentrato il {round(x,2)*100}% degli scambi di mail")

In 490 giorni, cioè il 6.0% del totale dei giorni, si è concentrato il 90.0% degli scambi di mail


In [98]:
# Trovare 10 persone più attive nel dataset
top10people = df["SenderFullName"].value_counts(normalize=True)
# print(df["SenderFullName"].value_counts(normalize=True)[:10])
su1 = df["SenderFullName"].value_counts()[:10].index.to_list()
i = 0
x = 0
# print(df["SenderFullName"].value_counts().index[i])

names = []
percs = []
while i < 10:
    for date in df["SenderFullName"].value_counts(normalize=True)[:10]:
        x += df["SenderFullName"].value_counts(normalize=True)[i]
        name_to_add = df["SenderFullName"].value_counts().index[i]
        perc_to_add = round(df["SenderFullName"].value_counts(normalize=True)[i]*100)
        names.append(name_to_add)
        percs.append(perc_to_add)
        i += 1

names.append("Others")
percs.append(10)

print(f"10 persone, cioè il {round(10/len(persons),2)*100}% del totale,"
      f" sono responsabili per il {round(x,2)*100}% del flusso di mail")

10 persone, cioè il 2.0% del totale, sono responsabili per il 89.0% del flusso di mail


In [99]:
print(top10people[:10])
import plotly.express as px

fig = px.pie(top10people[:10], values=top10people[:10], names=top10people[:10].index)
fig.show()

Hillary Clinton         0.256738
Huma Abedin             0.185300
Cheryl Mills            0.169826
Jake Sullivan           0.112315
Sidney Blumenthal       0.048098
Lauren Jiloty           0.043972
Philippe Reines         0.020503
Lona Valmoro            0.019987
Anne-Marie Slaughter    0.016763
Richard Verma           0.015216
Name: SenderFullName, dtype: float64


In [100]:
cum_sum = np.cumsum(top10people)

fig = go.Figure()

fig.add_trace(go.Bar(
    x=cum_sum.index[:TOP_K],
    y=top10people[:TOP_K],
    name="Markers and Text",
    marker_color="#636EFA",
    showlegend=False
))

fig.add_trace(go.Scatter(
    x=cum_sum.index[:TOP_K],
    y=cum_sum,
    mode="text+lines+markers",
    name = "Cumulative",
    text=[i for i in range(1, TOP_K + 1)],
    textposition="top center",
))

fig.update_xaxes(
    tickangle = 45,
    title_text = "Person",
)

fig.update_yaxes(
    title_text = "Activity probability",
)

fig.update_layout(
    title= dict(
        text=f'People activity',
        xanchor='center',
        x=0.5, y=.9),
)
fig.show()

In [101]:
''''''''''''
chart3 = px.treemap(df, path=['SenderFullName'],
                    height = 500,
                    title = "People activity")

chart3.update(layout=dict(title=dict(x=0.5)))
chart3.update_traces(textinfo = 'label + percent parent')

In [102]:
# Trovare i 5 giorni più intensi per le prime 10 persone più attive
names_group = df.groupby(["SenderFullName"])
for name in su1:
    print(name)
    print(names_group["DateSent"].value_counts().loc[name][:5],"\n")

Hillary Clinton
DateSent
2009-12-23    24
2009-09-20    21
2009-07-14    19
2009-12-29    17
2010-01-17    17
Name: DateSent, dtype: int64 

Huma Abedin
DateSent
2009-07-02    25
2009-08-21    17
2009-09-01    17
2009-08-26    15
2009-09-12    15
Name: DateSent, dtype: int64 

Cheryl Mills
DateSent
2010-01-24    17
2009-05-14    13
2009-06-02    13
2009-08-26    10
2010-01-13    10
Name: DateSent, dtype: int64 

Jake Sullivan
DateSent
2010-03-21    12
2010-04-02    12
2010-02-10    11
2009-09-12    9 
2010-08-28    9 
Name: DateSent, dtype: int64 

Sidney Blumenthal
DateSent
2009-11-05    12
2010-05-07    11
2010-01-23    8 
2010-05-11    8 
2009-11-29    7 
Name: DateSent, dtype: int64 

Lauren Jiloty
DateSent
2010-08-26    21
2010-10-08    7 
2009-09-21    6 
2010-08-24    6 
2010-08-25    6 
Name: DateSent, dtype: int64 

Philippe Reines
DateSent
2010-09-02    7
2009-08-07    5
2010-06-23    5
2010-08-08    5
2009-07-26    4
Name: DateSent, dtype: int64 

Lona Valmoro
DateSent
2009-

In [103]:
print(df["SenderFullName"].value_counts(normalize=True)[:11])
print(df["DateSent"].value_counts(normalize=True)[:11])
p = df.groupby('SenderFullName').DateSent.value_counts(normalize=True)
p.head()

Hillary Clinton         0.256738
Huma Abedin             0.185300
Cheryl Mills            0.169826
Jake Sullivan           0.112315
Sidney Blumenthal       0.048098
Lauren Jiloty           0.043972
Philippe Reines         0.020503
Lona Valmoro            0.019987
Anne-Marie Slaughter    0.016763
Richard Verma           0.015216
Melanne Verveer         0.012379
Name: SenderFullName, dtype: float64
2009-12-23    0.006061
2009-09-20    0.006061
2010-03-21    0.005932
2010-02-10    0.005803
2009-07-14    0.005674
2009-12-24    0.005674
2010-02-06    0.005674
2010-01-17    0.005545
2009-08-26    0.005416
2009-08-29    0.005416
2009-10-18    0.005287
Name: DateSent, dtype: float64


SenderFullName                 DateSent  
AGNA USEMB Kabul Afghanistan   2009-05-29    1.000000
American Beverage Association  2010-10-18    1.000000
Andrew Shapiro                 2009-07-31    0.285714
                               2009-08-14    0.142857
                               2009-08-16    0.142857
Name: DateSent, dtype: float64